# Full Completiontools example
A working sample going over the full process from start to finish.

The completion process is aimed at complementing existing large datasets with newer smaller datasets. 
It also aims to leave as much of the original dataset in tact as possible, assuming it's more detailed and precise.

## Importing the package and setting up

In [1]:
from context import completiontools
import completiontools.utils as utils
import os
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
tresholdResolution = 0.05 # The max coverage distance
ogGeometryPath = os.path.join(os.getcwd(),"localfiles/voxel_grond_pointcloud.ply")
newGeometryPath =  os.path.join(os.getcwd(),"localfiles/GrondSampleMesh.obj")

## Getting the Geometries
Geometries can be either directly imported from a file or retrieved from a `geomapi.GeometryNode`

In [3]:
ogGeometry = completiontools.utils.get_geometry(ogGeometryPath)
newGeometry = completiontools.utils.get_geometry(newGeometryPath)

In [4]:
utils.show_geometries([ogGeometry, newGeometry])

## Single function
`combine_geometry()` is a compound function which performs the full algorithm ans returns the combined geometry.

In [5]:
combinedGeometry = completiontools.combine_geometry(ogGeometry, newGeometry, tresholdResolution)

In [6]:
utils.show_geometries([combinedGeometry])

## Step-by-step

The combination algorithm is performed in 2 phases: the removal and the addition phase.
In the removal phase, all the out-of-date points in the original mesh are removed to make room for the new points.
In the addition phase, only the new (uncovered) points from the new geometry are added, this is to ensure the existing original pointcould can keep as much relevant data as possible.

### Removal Phase Step 1: Create a convex hull of the newGeometry
In order to prevent false removal of the original geometry, we need to limit the evaluated points of the original geometry. This is why a convex hull is created to encapsulate all the relevant points.


In [7]:
newGeoHull = completiontools.get_convex_hull(newGeometry)

In [8]:

utils.show_geometries([utils.get_lineset(newGeoHull), newGeometry])

### Removal Phase Step 2: Filter out the irrelevant points in the ogGeometry
A subselection of the original geometry is made with the convex hull as boundary volume.


In [9]:
relevantOg, irrelevantOg = completiontools.get_points_in_hull(ogGeometry, newGeoHull)

In [10]:
utils.show_geometries([utils.get_lineset(newGeoHull), relevantOg])

### Removal Phase Step 3: Isolate the not covered points of the ogGeometry compared to the newGeometry
To determine which points are still relevant and therefor, should not be removed we perform 2 Checks, the first one being the Coverage check. This checks If the original points are also captured on the new dataset. if they are not, they are either no longer up-to-date and should be removed, or they were not visible to the scanner and should remain in the scan. This is where the second check comes in.

In [11]:
newGeometryPoints = newGeometry.sample_points_poisson_disk(number_of_points=100000)
coveredPoints, unCoveredPoints = utils.filter_pcd_by_distance(relevantOg, newGeometryPoints, tresholdResolution)

In [12]:
utils.show_geometries([unCoveredPoints, newGeometry])

### Removal Phase Step 4: performm the visibility check
The uncovered points are chacked agains the new mesh. assuming the new scanner has caoptured everything it can see, Points that are hidden behind the geometry were not visible during the scanning process. This check is performed by finding the closest faces to the points and comparing the normal direction. Points facing the faces could have been seen by the scanner and vise versa.

In [13]:
insideList, outsideList = completiontools.check_point_inside_mesh(unCoveredPoints.points, newGeometry)
visiblePoints = o3d.geometry.PointCloud()
visiblePoints.points = o3d.utility.Vector3dVector(outsideList)
invisiblePoints = o3d.geometry.PointCloud()
invisiblePoints.points = o3d.utility.Vector3dVector(insideList)

In [14]:
utils.show_geometries([visiblePoints, invisiblePoints, newGeometry], True)

### Addition Phase Step 5: Filter the newGeometryPoints to only keep the changed geometry
Because we assume The original geometry is of better quality, we will only add points that are changed. Therefor we apply an inverted distance query from the new points to the existing geometry.


In [15]:
existingNewGeo, newNewGeo = utils.filter_pcd_by_distance(newGeometryPoints, relevantOg, tresholdResolution)

### Addition Phase Step 6: Combine the irrelevant, unchanged and changed geometry
The final step is combining the original irrelevant data, the unganged original data and the changed new geometry. The resulting geometry is a combination of both, aimed at retaining as much of the original as possible.


In [16]:
newCombinedGeometry = coveredPoints + invisiblePoints + newNewGeo

In [17]:
utils.show_geometries([coveredPoints, invisiblePoints, newNewGeo], True)